In [1]:
import pandas as pd
%matplotlib inline

In [2]:
#read data from csv files
how_much = pd.read_csv('items_number.csv', sep = ';')
items = pd.read_csv('items.csv', sep = ';')
paths = pd.read_csv('paths.csv', sep = ';')
machines = pd.read_csv('machines.csv', sep = ',')
norm_times = pd.read_csv('norm_time.csv', sep = ';')
setup_times = pd.read_csv('setup_time.csv', sep = ';')

In [3]:
#some setups to correct data
setup_times.fillna(value = 10, inplace = True) #fill blank fields
norm_times.fillna(value = '-', inplace = True)
paths.fillna(value = -1, inplace = True)
how_much = how_much[0:72] #cut useless definitions

pd.options.display.max_rows = 100
N_DETAILS = 72
N_MACHINES = 10

In [4]:
machines.head(15)
unique_machines = {'Пила' : 0, 'Плазменная' : 1, 'Токарный' : 2, 'Фрезерный' : 3, 
                   'Сверление' : 4, 'Сварка' : 5, 'Окраска' : 6}
paths.replace(unique_machines, inplace = True)
machines.head(10)

,№,Название,Тип
0,1,Станок №1,Пила
1,2,Станок №2,Плазменная
2,3,Станок №3,Токарный
3,4,Станок №4,Токарный
4,5,Станок №5,Фрезерный
5,6,Станок №6,Фрезерный
6,7,Станок №7,Сверление
7,8,Станок №8,Сверление
8,9,Станок №9,Сварка
9,10,Станок №10,Окраска


In [5]:
class step:
    def __init__(self, machine, time, setup):
        self.machine_ = machine
        self.time_ = time
        self.setup_ = setup

In [6]:
class detail_type:
    def __init__(self, n, path, n_steps): #path - is an array of class step
        self.number_ = n
        self.n_steps_ = n_steps
        self.path_ = path

In [7]:
class dtype_state:
    def __init__(self, detail_type, n_to_do):
        self.detail_type_ = detail_type
        self.n_to_do_ = n_to_do
        self.n_done_ = 0
        self.details_progress_ = [detail(detail_type.n_steps_) for i in range(n_to_do)] #array of details
        self.needed_details_ = [] #-1 if simple detail; array of 'sub'-details if complex

In [8]:
class detail:
    def __init__(self, n_steps):
        self.progress_ = [0 for i in range(n_steps)]    #array the same size as detail_type.route - 
                                                        #minutes spent on every machine
        self.done_ = [0 for i in range(n_steps)]        #boolean array to check if work on the machine is done

In [9]:
State = [] #array of dtype_states

for i in range(N_DETAILS):        # in this cycle we initialize State
    #make path for the type
    j = 2
    path = []
    machine_type = paths.iloc[i + 1, j]
    while machine_type != -1:
        one_step = step(machine_type, 
                    norm_times.iloc[i, machine_type + 2],
                   setup_times.iloc[i, machine_type + 2])
        path.append(one_step)
        j += 1
        if j > 6:
            break
        machine_type = paths.iloc[i + 1, j]
        
    this_type = detail_type(i, path, j - 2)
    
    n_to_do = how_much['Кол-во'][i]
    state_elem = dtype_state(this_type, int(n_to_do))
    State.append(state_elem)
    
for i in range(N_DETAILS):
    rest = int ((how_much.iloc[i, 0] * 10) % 10)
    if rest != 0:
        State[i - rest].needed_details_.append(i)
    
# print(State[71].detail_type_.path_[0].machine_)
# print("hi")


2
hi


In [10]:
class machine:
    def __init__(self, mtype):
        self.machine_type_ = mtype
        self.processing_detail_ = [-1, -1] #[0] - type of the detail and [1] - number in details_progress array
    def what_available():
        pass

In [11]:
Factory = [] #array of machines

for i in range(N_MACHINES):
    mtype = unique_machines[machines.iloc[i, 2]]
    Factory.append(machine(mtype))